# Практическое задание №12 по теме "Модель Transformer-2".

Загрузка библиотек:

In [1]:
import pandas as pd
import numpy as np
import json
import random
import torch

In [2]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import razdel

Загрузка и преобразование данных:

In [7]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [8]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [9]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

Функции метрик:

In [10]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [11]:
def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


Загрузка модели и функция суммаризации:

In [12]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
mod = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(mod)
tokenizer = T5Tokenizer.from_pretrained(mod)
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):

    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

Тестирование суммаризации:

In [13]:
summarize_pred_test = summarize(test_records[0]['text'], len(test_records[0]['summary'].split()))

In [14]:
summarize_pred_test

'Американское аэрокосмическое агентство NASA объявило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации.'

In [15]:
test_records[0]['summary']

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

Создание датасета из данных:

In [16]:
df_txt = pd.DataFrame(test_records)

In [17]:
df_summ = df_txt[:10]

In [18]:
df_summ['summary_pred'] = df_summ.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

<ipython-input-18-66f2bf1d4ad8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summ['summary_pred'] = df_summ.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [19]:
df_summ

,url,text,title,summary,date,summary_pred
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11,Американское аэрокосмическое агентство NASA об...
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13,В Бурятии прошел праздничный концерт «Танцуют ...
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08,В Белоруссии впервые прошли выборы членов сове...
3,https://www.gazeta.ru/culture/2020/03/01/a_129...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,2020-03-01 16:50:06,"Надежда Бабкина рассказала, как ей удалось сбр..."
4,https://www.gazeta.ru/business/2020/02/06/1294...,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,2020-02-06 12:41:24,В Верховной Раде раскритиковали законопроект о...
5,https://www.gazeta.ru/army/2019/11/13/12810158...,Журналист Питер Суппли Бенсон датского издания...,«Новый кулак в Арктике»: в Дании испугались «И...,Датского журналиста обеспокоила активность Рос...,2019-11-13 13:03:37,Журналист Питер Суппли Бенсон назвал «новым ку...
6,https://www.gazeta.ru/politics/2019/10/03_a_12...,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,2019-10-03 20:46:08,На заседании Парламентской ассамблеи Совета Ев...
7,https://www.gazeta.ru/politics/2020/01/03_a_12...,Самые известные российские пранкеры Владимир «...,Именем Греты Тунберг: Вован и Лексус разыграли...,Пранкеры Вован и Лексус разыграли члена конгре...,2020-01-03 19:25:16,Российские пранкеры Владимир «Вован» Кузнецов ...
8,https://www.gazeta.ru/culture/2019/11/06/a_127...,Российский актер и театральный педагог Виталий...,Умер актер сериала «Улицы разбитых фонарей»,Актер сериала «Улицы разбитых фонарей» Виталий...,2019-11-06 14:17:33,Российский актер Виталий Жигалин ушел из жизни...
9,https://www.gazeta.ru/business/2020/01/23/1292...,Госдума одобрила в первом чтении внесенные пре...,Дождались: работающим пенсионерам восстановят ...,Работающие пенсионеры дождались индексации. Эт...,2020-01-23 19:32:12,Госдума одобрила в первом чтении поправки в Ко...


Функция подсчета метрик для датафрейма:

In [20]:
def calc_lead_n_score_df(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in records.iterrows():
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

Результаты:

In [29]:
calc_lead_n_score_df(df_summ.iloc[1:2], n=10)

Count: 1
Ref: 25 и 26 февраля в кремлевском дворце съездов праздновали сагаалган — восточный новый год. бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре бурятии. в кремле выступил бурятский государственный академический театр оперы и балета, национальный цирк, бурятский национальный театр песни и танца «байкал», ставший победителем шоу «танцуют все!» на телеканале «россия», а также другие профессиональные и самодеятельные коллективы региона. более 300 артистов из одного региона на главной сцене страны - похоже это рекорд россии. зрителям рассказали, что республике бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. и все они были представлены в москве. как писали после шоу зрители в соцсетях: «а мы думали, ч

In [30]:
calc_lead_n_score_df(df_summ.iloc[1:2], summary_col = 'summary_pred', n=10)

Count: 1
Ref: в бурятии прошел праздничный концерт «танцуют все!» на телеканале «россия»
Hyp: около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре бурятии. в кремле выступил бурятский государственный академический театр оперы и балета, национальный цирк, бурятский национальный театр песни и танца «байкал», ставший победителем шоу «танцуют все!» на телеканале «россия», а также другие профессиональные и самодеятельные коллективы региона. более 300 артистов из одного региона на главной сцене страны - похоже это рекорд россии. зрителям рассказали, что республике бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. и все они были представлены в москве. как писали после шоу зрители в соцсетях: «а мы думали, что в бурятии только буряты живут…». для неподготовленного зрителя это вообще были вечера открытий. например, когда еще в кремлевско

**Вывод:** как видно из метрик, сгенерированный заголовок второй новости из датафрейма показал результат заметно хуже.

---